#### Test de modèles

* 1ère étape : chargement des données
* 2ème étape : tester les modèles possibles sur les données
* 3ème étape : fixer et évaluer les métriques des modèles pour comparaison
* 4ème étape : améliorations potentielles

In [2]:
# Importations

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

1. Chargement des données

In [6]:

X_train = pd.read_csv("X_train.csv").drop(columns=["Unnamed: 0", "index"])
X_test = pd.read_csv("X_test.csv").drop(columns=["Unnamed: 0", "index"])
y_train = pd.read_csv("y_train.csv").drop(columns=["Unnamed: 0", "index"])
y_test = pd.read_csv("y_test.csv").drop(columns=["Unnamed: 0", "index"])

X_test

,Popularity,Vote_Count,Release_Year,Release_Month,Release_Day,Release_Weekday,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,cn,de,en,es,fr,it,ja,ko,ru,zh,Other_languages
0,-0.968122,-1.713799,1.012133,0.954649,0.124506,0.020538,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,-0.820175,0.256461,-3.273022,0.360292,0.570560,0.020538,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1.978239,-0.796795,0.820260,0.828493,0.168183,0.920735,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1.516214,1.675395,0.308600,0.231714,0.828493,0.570560,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,-0.721243,0.329474,-2.505532,0.994679,0.954649,0.570560,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,0.870216,1.222367,0.116727,0.828493,0.920735,0.954649,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1470,3.638098,-0.169552,0.948175,0.227989,0.168183,0.020538,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1471,-0.649253,0.759024,-4.808003,0.954649,0.920735,0.828493,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1472,-0.963255,-0.124953,0.628387,0.360292,0.356048,0.121599,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


2. Test des modèles

Etant donné que nous sommes face à un problème de régression, avec des données d'entraînement en faible nombre (< 10K), nous testerons :
* La Régression Linéaire multiple,
* Les Régressions Linéaires pénalisées de Ridge et de Lasso,
* Le modèle ElasticNet, qui combine ces deux dernières.

In [7]:
# Définition des variables
feats = list(X_train.columns)

# Régression linéaire
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
lr_prediction = model_lr.predict(X_test)
# Coefficients
lr_coeffs = list(model_lr.coef_)


# Régression de Lasso
model_lasso = LassoCV(cv=8)
model_lasso.fit(X_train, y_train)
lasso_prediction = model_lasso.predict(X_test)
# Coefficients
lasso_coeffs = list(model_lasso.coef_)
# Alphas
lasso_alphas = model_lasso.alphas_


# Régression de Ridge
model_ridge = RidgeCV(cv=8)
model_ridge.fit(X_train, y_train)
ridge_prediction = model_ridge.predict(X_test)
# Coefficients
ridge_coeffs = list(model_ridge.coef_)



# ElasticNet
model_en = ElasticNetCV(l1_ratio=(0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99),
                        alphas=(0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0),
                        cv = 8)
model_en.fit(X_train, y_train)
en_prediction = model_en.predict(X_test)
# Coefficients
en_coeffs = list(model_en.coef_)
# Alphas
lasso_alphas = model_en.alphas_



c:\Users\ambro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1664: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ambro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1664: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3. Métriques et comparaisons

L'objectif ici est de réussir à prédire le vote moyen des utilisateurs pour la sortie d'un nouveau film à partir des autres variables utilisateur.

Nous utiliserons donc :
* Le coefficient de détermination (R²) pour savoir comment le modèle s'ajuste aux données ;
* La Moyenne des valeurs Absolues des Erreurs (MAE) en raison du nombre élevé d'outliers dans les variables numériques.

In [9]:
model_list = [model_lr, model_lasso, model_ridge, model_en]
for model in model_list :
    print(f"{model} :")
    print("\nScore pour l'ensemble test : ", model.score(X_test, y_test))
    print("\nRMSE pour l'ensemble test : ", mean_squared_error(model.predict(X_test), y_test))
    print("\nScore pour l'ensemble train : ", model.score(X_train, y_train))
    print("\nRMSE pour l'ensemble train : ", mean_squared_error(model.predict(X_train), y_train), "\n")


LinearRegression() :

Score pour l'ensemble test :  0.42436572240811854

RMSE pour l'ensemble test :  0.7417127213116776

Score pour l'ensemble train :  0.4130539656119828

RMSE pour l'ensemble train :  0.7478386580356045 

LassoCV(cv=8) :

Score pour l'ensemble test :  0.42390334056237966

RMSE pour l'ensemble test :  0.7423085067095223

Score pour l'ensemble train :  0.41286999698236326

RMSE pour l'ensemble train :  0.7480730558252391 

RidgeCV(cv=8) :

Score pour l'ensemble test :  0.4243299689984247

RMSE pour l'ensemble test :  0.7417587900741411

Score pour l'ensemble train :  0.41305214652412214

RMSE pour l'ensemble train :  0.7478409757686582 

ElasticNetCV(alphas=(0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0),
             cv=8, l1_ratio=(0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99)) :

Score pour l'ensemble test :  0.42392729737539914

RMSE pour l'ensemble test :  0.7422776380248899

Score pour l'ensemble train :  0.4129116003116323

RMSE pour l'ensemble tra